# Example of Gaussian plume inversion with ddeq

In [ ]:
import os

import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import xarray as xr

# import and setup matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import ddeq
import ucat

from ddeq import DATA_PATH
from ddeq.smartcarb import DOMAIN

CRS = ccrs.UTM(32)
WGS84 = ccrs.PlateCarree()

# list of point sources
sources = ddeq.misc.read_point_sources()
sources

In [ ]:
overlapping_sources = ['Janschwalde', 'Berlin', 'Boxberg']
pixel_size = 2e3

np.where(
        sources["diameter"].sel(source=overlapping_sources).values <= pixel_size,
        0.0,
        10e3
)

In [ ]:
PRIORS = {
    'Janschwalde': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
    'Schwarze Pumpe': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
    'Boxberg': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
}

In [ ]:
q = 0.995 # default of 0.995 for non-overlapping plumes
          #          or 0.990 for testing overlapping plumes of Schwarze Pumpe and Boxberg (experimental)

filename = os.path.join(ddeq.DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')

data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='medium',
                                  no2_noise_scenario='high')

data = ddeq.dplume.detect_plumes(data, sources.sel(source=['Janschwalde', 'Schwarze Pumpe', 'Boxberg']),
                                 variable='NO2', variable_std='NO2_std', crs=CRS,
                                 filter_type='gaussian', filter_size=0.5, q=q)

ddeq.vis.show_level2(data, 'NO2', gas='NO2', domain=DOMAIN, winds=None,
                    do_zoom=True, show_clouds=True);

In [ ]:
data = ddeq.curves.fit_to_detections(data, n_nodes=3, force_origin=True, use_weights=True)
data = ddeq.curves.compute_natural_coords(data)
data = ddeq.curves.compute_plume_areas(data)

for gas in ["CO2", "NO2"]:
    data = ddeq.background.estimate(data, gas)
    data = ddeq.emissions.compute_plume_signal(data, gas)

    for variable in [
        gas,
        f"{gas}_estimated_background",
        f"{gas}_minus_estimated_background",
    ]:
        ddeq.emissions.convert_units(data, gas, variable)

ddeq.vis.show_level2(data, 'NO2', gas='NO2', domain=DOMAIN, winds=None,
                    do_zoom=True, show_clouds=True, crs=CRS);

In [ ]:
time = pd.Timestamp(data.time.values)
winds = ddeq.wind.read_smartcarb(time, sources.lon, sources.lat, radius=0.05, data_path=DATA_PATH)

In [ ]:
%%time

# takes a few minutes
data, results = ddeq.gauss.estimate_emissions(data, winds, sources, priors=PRIORS,
                                              gases=['NO2', 'CO2'], fit_decay_times=[True, False])

In [ ]:
fig = ddeq.vis.plot_gauss_result(data, results, ['Janschwalde', 'Schwarze Pumpe', 'Boxberg'],
                                 'CO2', crs=CRS, vmin=-20e-3, vmax=60e-3)

In [ ]:
fig = ddeq.vis.plot_gauss_result(data, results, ['Janschwalde', 'Schwarze Pumpe', 'Boxberg'],
                                 'NO2', crs=CRS, vmin=-20e-6, vmax=60e-6)

Show estimated emissions

In [ ]:
for name in ['Janschwalde', 'Schwarze Pumpe', 'Boxberg']:
    print(name)
    print('CO2', '%.2f Mt/a' % ucat.convert_mass_per_time_unit(results.sel(source=name).CO2_emissions, 'kg/s', 'Mt/a'))
    print('NOx', '%.2f kt/a' % ucat.convert_mass_per_time_unit(results.sel(source=name).NO2_emissions, 'kg/s', 'kt/a'))
    print()

#Janschwalde
#CO2 54.81 Mt/a
#NOx 40.69 kt/a

#Schwarze Pumpe
#CO2 15.08 Mt/a
#NOx 8.93 kt/a

#Boxberg
#CO2 nan Mt/a
#NOx 19.55 kt/a